<h1> Artificial Neural Networks </h1>

In [2]:
# import libraries

import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.1.0'

<h2> 1. Data Preprocessing </h2>

In [24]:
# Data Preprocessing Template

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('../sample_data/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


<h3> Encode Catigorical data </h3>

In [25]:
#label encoding fpr binary  variable Gender
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:,2])

In [26]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [27]:
# One Hot encoding for Variabel Country

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

<h3> Split Dataset into Training dataset and Validation set </h3>

In [29]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


<h3> Feature Scaling </h3>
Feature Scaling is ABSOLUTELY necessary for Deeep Learning. this is why it will be applied to all features

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

<h2> Build the Artificial Neural Network </h2>

- Initialize ANN
- Adding the input layer and first hidden layer
- Adding the second hidden layer
- Adding the output layer

Initialize ANN

In [33]:
ann = tf.keras.models.Sequential()

Adding th input layer and first hidden layer

In [34]:
# units is number of neurons in first hidden layer
# no rule of thumb for hyperparameter. experiment for better results
# relu is the rectified linear units activation function
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

Adding second hidden layer

In [35]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

Adding Output layer

In [36]:
#add math can be used to add any kind of layer, including the output layer

# activation function "sigmoid" is used because it returns the probability of the result being 1 or 0

# for multiple classes in the output data the activation function has to softmax
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

<h2> Training the ANN </h2>

Compiling the ANN

In [38]:
# ADAM ist chose because it can perform Stochastic gradient descent, which will update the weights to reduce the loss between predictions and real values

# for binary values loss funtion ALWAYS has to be binary_crossentropy
# for multiple classes it has to catigorical_crossentropy

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Train ANN

In [39]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Train on 8000 samples
Epoch 1/100
8000/8000 [==============================] - 1s 145us/sample - loss: 0.5281 - accuracy: 0.7924
Epoch 2/100
8000/8000 [==============================] - 0s 50us/sample - loss: 0.4707 - accuracy: 0.7976
Epoch 3/100
8000/8000 [==============================] - 0s 40us/sample - loss: 0.4526 - accuracy: 0.8014
Epoch 4/100
8000/8000 [==============================] - 0s 41us/sample - loss: 0.4431 - accuracy: 0.8076
Epoch 5/100
8000/8000 [==============================] - 0s 40us/sample - loss: 0.4359 - accuracy: 0.8112
Epoch 6/100
8000/8000 [==============================] - 0s 39us/sample - loss: 0.4298 - accuracy: 0.8163
Epoch 7/100
8000/8000 [==============================] - 0s 40us/sample - loss: 0.4248 - accuracy: 0.8196
Epoch 8/100
8000/8000 [==============================] - 0s 43us/sample - loss: 0.4203 - accuracy: 0.8217
Epoch 9/100
8000/8000 [==============================] - 0s 44us/sample - loss: 0.4160 - accuracy: 0.8255
Epoch 10/100
8000/8000 

<h2> Making the prediction and evelute the model </h2> 

In [44]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

array([[0.03889995]], dtype=float32)

In [45]:
# trans form resulst (probaility) into prediciton if costumer wil leave bank or not
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


<h3> Predicting the test set </h3> 

In [47]:
y_pred = ann.predict(X_test)
#transform probabilities to binary results

y_pred = (y_pred > 0.5)

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


<h3> Creating Confusion Matrix </h3> 

In [50]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1508   87]
 [ 196  209]]


0.8585